<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-6.-Design-Patterns-with-First-Class-Functions" data-toc-modified-id="Chapter-6.-Design-Patterns-with-First-Class-Functions-1">Chapter 6. Design Patterns with First-Class Functions</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Implementation-of-Strategy-pattern-based-on-abstract-classes-and-polymorphism" data-toc-modified-id="Implementation-of-Strategy-pattern-based-on-abstract-classes-and-polymorphism-1.0.1">Implementation of Strategy pattern based on abstract classes and polymorphism</a></span></li><li><span><a href="#Strategies-pattern-implemented-as-functions-as-first-class-objects" data-toc-modified-id="Strategies-pattern-implemented-as-functions-as-first-class-objects-1.0.2">Strategies pattern implemented as functions as first class objects</a></span></li><li><span><a href="#Finding-Strategies-in-a-Module-using-inspect.getmembers" data-toc-modified-id="Finding-Strategies-in-a-Module-using-inspect.getmembers-1.0.3">Finding Strategies in a Module using inspect.getmembers</a></span></li><li><span><a href="#Implementation-of-Command-pattern-based-on-callable-objects" data-toc-modified-id="Implementation-of-Command-pattern-based-on-callable-objects-1.0.4">Implementation of Command pattern based on callable objects</a></span></li></ul></li></ul></li></ul></div>

# Chapter 6. Design Patterns with First-Class Functions

### Implementation of Strategy pattern based on abstract classes and polymorphism


In [154]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # the Strategy: an abstract base class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


In [155]:
joe = Customer('John Doe', 0) 
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),   
         LineItem('apple', 10, 1.5),
         LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo())   #<Order total: 42.00 due: 42.00>
Order(ann, cart, FidelityPromo())   #<Order total: 42.00 due: 39.90>

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, BulkItemPromo()) #<Order total: 30.00 due: 28.50>

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

Order(joe, long_order, LargeOrderPromo())   #<Order total: 10.00 due: 9.30>
Order(joe, cart, LargeOrderPromo()) #<Order total: 42.00 due: 42.00>

<Order total: 42.00 due: 42.00>

### Strategies pattern implemented as functions as first class objects

In [156]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)   
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):   
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [157]:
Order(joe, cart, fidelity_promo)   #<Order total: 42.00 due: 42.00>
Order(ann, cart, fidelity_promo)  #<Order total: 42.00 due: 39.90>
Order(joe, banana_cart, bulk_item_promo) #<Order total: 30.00 due: 28.50>
Order(joe, long_order, large_order_promo)   #<Order total: 10.00 due: 9.30>
Order(joe, cart, large_order_promo) #<Order total: 42.00 due: 42.00>

<Order total: 42.00 due: 42.00>

### Finding Strategies in a Module using inspect.getmembers

In [159]:
import inspect
class Promotions:
    def fidelity_promo(self, order):   
        """5% discount for customers with 1000 or more fidelity points"""
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


    def bulk_item_promo(self, order):
        """10% discount for each LineItem with 20 or more units"""
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


    def large_order_promo(self, order):
        """7% discount for orders with 10 or more distinct items"""
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0
    
promotions = Promotions()
promos = [func for name, func in inspect.getmembers(promotions, 
                                                    inspect.ismethod)]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)


In [160]:
Order(joe, cart, best_promo)

<Order total: 42.00 due: 42.00>

### Implementation of Command pattern based on callable objects


In [161]:
class MacroCommand:
    """A command that executes a list of commands"""

    def __init__(self, commands):
        self.commands = list(commands)  # 

    def __call__(self):
        for command in self.commands:  # 
            command()